In [1]:
secret_key = " sk-proj-pqEfsvNNHPlhNAvPbJu3T3BlbkFJqfdIWBNRHYBPdTSNDYLZ"
secret_key

' sk-proj-pqEfsvNNHPlhNAvPbJu3T3BlbkFJqfdIWBNRHYBPdTSNDYLZ'

In [2]:
import os
import openai
os.environ['OPENAI_API_KEY'] = secret_key
openai.api_key = os.getenv('OPENAI_API_KEY')

In [3]:
# import logging
# logging.basicConfig(level=logging.DEBUG)

In [4]:
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import Docx2txtLoader
from langchain.document_loaders import UnstructuredExcelLoader
from langchain.llms import OpenAI
from langchain_core.vectorstores import VectorStoreRetriever
from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document
from langchain.embeddings.openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings()

c:\Users\a_rah\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [5]:
client = openai.OpenAI(api_key=secret_key)

In [6]:
pdf_directory = 'project_files/questions_folder/'
questions = []

In [7]:
for filename in os.listdir(pdf_directory):
    if filename.endswith('.pdf'):
        file_path = os.path.join(pdf_directory, filename)
        question = PyPDFLoader(file_path).load()
        questions.append(question)
questions = questions[:len(questions)//2]

In [8]:
rubric = PyPDFLoader('project_files/rubric.pdf').load()

In [9]:
chunk_size = 1000
chunk_overlap = 50

In [10]:
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap, 
    separators=["\n\n", "\n", ".", " ", ""]
)

In [11]:
persist_directory = 'content/vectorstore/chroma/'

In [12]:
splits = [r_splitter.split_documents(q) for q in questions]
# logging.debug(f"Total document splits: {len(splits)}")
questions = splits[0]
for i in splits:
    if i != splits[0]:
        questions += i
questions += r_splitter.split_documents(rubric)

In [13]:
vectordb = Chroma.from_documents(
    documents = questions,
    embedding = embedding,
    persist_directory = persist_directory
)

In [14]:
llm = OpenAI(model='gpt-3.5-turbo-instruct', temperature=0)
retriever = VectorStoreRetriever(vectorstore=vectordb)
qachain = RetrievalQA.from_llm(llm=llm, retriever=retriever)

c:\Users\a_rah\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [15]:
system_context = "You are a python teaching assistant. Your goal is to help students with their assignments and if they give you code you grade them based on the rubric pdf in the database and the questions of that assignment number in the database."

# Read the student's code file
student_file_path = 'project_files/student_folder/Assignment_2_Abdelrahman_Amr_Eweis_202100752.py'
with open(student_file_path, 'r') as file:
    student_code = file.read()

# Format the query with the system context
assignment_number = 2
# query = f"{system_context}\n\nstudent code:{student_code}\n\nWhat is the grade of this assignment{assignment_number}?"
query = "what are the questions in assignment 2?"

# Run the QA Chain
response = qachain({'query': query})
print(response['result'])

c:\Users\a_rah\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


 I don't know, as I don't have access to the assignment.
